In [218]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [219]:
from __future__ import absolute_import, division, unicode_literals

import sys, os
import io
import numpy as np
import logging
import torch
import code
import argparse

sys.path.insert(0, "../")
import discoeval 
from examples.bert import batcher, prepare
from discoeval.so import SPEval, BSOEval, DCEval, SSPEval

from transformers import BertConfig, BertTokenizer, BertModel, BertForPreTraining

In [116]:
# Set up logger
PATH_TO_DATA = "../data/"
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)
config = BertConfig.from_pretrained('bert-{}-uncased'.format("base"))
config.output_hidden_states = True
config.output_attentions = True
tokenizer = BertTokenizer.from_pretrained('bert-{}-uncased'.format("base"))

# model = BertModel.from_pretrained('../models/conpono/bert/', config=config).cuda()
# model.eval()

# Set params for SentEval
params_discoeval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10, 'batch_size': 8,
                    'tokenizer': tokenizer, "layer": "pooler", "model": "base", "seed": 111}
params_discoeval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                 'tenacity': 5, 'epoch_size': 4}
se = discoeval.engine.SE(params_discoeval, batcher, prepare)


transfer_tasks = [
    ['SParxiv'],
    ['BSOwiki'],
    ['DCwiki']]
    #['SParxiv', 'SProc', 'SPwiki'], 
    #['DCchat', 'DCwiki'],
    #['BSOarxiv', 'BSOroc', 'BSOwiki'], 
    #['SSPabs', 'RST'],
    #[ 'PDTB-E', 'PDTB-I']]
    
#se.evaluation = SPEval(os.path.join(PATH_TO_DATA, 'SP/arxiv'), "Sentence Position arXiv", nclasses=5, seed=111)
#se.params.current_task = transfer_tasks[0][0]
#se.evaluation.do_prepare(se.params, se.prepare)
results = se.eval(transfer_tasks[1])
print(results)

2020-04-23 18:28:31,602 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-04-23 18:28:32,156 : https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-config.json HTTP/1.1" 200 0
2020-04-23 18:28:32,165 : loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/daniter/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
2020-04-23 18:28:32,181 : Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "outpu

DANITER False
DANITER cuda


2020-04-23 18:28:41,313 : [('reg:1e-09', 70.53)]
2020-04-23 18:28:41,315 : Validation : best param found is reg = 1e-09 with score             70.53
2020-04-23 18:28:41,319 : Evaluating...


DANITER False
DANITER cuda


2020-04-23 18:28:48,342 : 
Task: BSO wiki, Dev acc : 70.53 Test acc : 72.7 for             Binary Sentence Ordering prediction



[[2615 1385]
 [ 799 3201]]
{'BSOwiki': {'devacc': 70.53, 'acc': 72.7, 'ndev': 8000, 'ntest': 8000}}


In [117]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").xravel()

conpono_true = np.argwhere(conpono_pred == labels) 
bert_false = np.argwhere(bert_pred != labels) 
improvement = np.intersect1d(conpono_true, bert_false)

In [121]:
markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])

#markers = set(["but", "when", "if", "before", "because",
     #     "while", "though", "after", "so", "although", "then",
     #     "also", "still"])

In [122]:
import random
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

for i in random.sample(list(improvement), 4):
    # print(i)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set(sents[0] + sents[1])
            if words.intersection(markers):
                sents = [" ".join(s) for s in sents]
                print(sents[0])
                print("-")
                print(sents[1])
            
    print("~"*20)

The district lies east of the Melbourne and nestled between the Strzelecki Ranges to the south and the Baw Baw Ranges , part of the Great Dividing Range , to the north .
-
Mount Baw Baw ( ) is the highest peak to the north of the Latrobe Valley , due north of Moe .
~~~~~~~~~~~~~~~~~~~~
He was briefly imprisoned by Henry VII , but later restored to his position .
-
A few years later he was murdered by citizens of York during a revolt against Henry VII 's taxation .
~~~~~~~~~~~~~~~~~~~~
Oncken baptized these believers and established a church in Copenhagen in that year .
-
Until 1849 , when religious liberty was granted through the Constitution of 1849 , Baptists were fined , imprisoned , and their infants baptized by compulsion .
~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~


In [146]:
import random
from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
print(len(improvement))
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(improvement):# random.sample(list(improvement), 4):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[0] + sents[1]])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
            break

1241
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [147]:
for w, c in word_counter.most_common():
    print(w, "\t\t",c/ num_words)

and 		 0.01954220376931939
as 		 0.009173314783818446
also 		 0.0030432797861009065
but 		 0.0018911810099341347
after 		 0.0017172793078712258
when 		 0.001652066169597635
before 		 0.0011520987761667718
then 		 0.000847770797556681
while 		 0.0008042953720409538
although 		 0.000739082233767363
so 		 0.00047822968067299956
because 		 0.00045649196791513596
still 		 0.0002608525530943634
if 		 0.00023911484033649978
though 		 0.00023911484033649978


In [148]:
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
    if j in indexes_to_skip:
        continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1


In [149]:
for w, c in general_word_counter.most_common():
    print(w, "\t\t",c/ general_num_words)

and 		 0.019658636272054802
as 		 0.008958639150332442
also 		 0.0027487551449212793
after 		 0.0017485536654865729
but 		 0.0014103560429439023
when 		 0.0013383991019773767
before 		 0.0008778746797916127
then 		 0.0007339607978585614
while 		 0.0006404167746020781
although 		 0.000575655527732205
because 		 0.00038856748121923843
so 		 0.00029502345796275506
if 		 0.00028063206976944993
still 		 0.0002734363756727974
though 		 0.00025184929338283973


In [150]:
counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc

and 		 -0.005922715142805883
as 		 0.023962973603869057
also 		 0.107148372863914
but 		 0.34092452710493154
after 		 -0.01788584373053501
when 		 0.2343598909748523
before 		 0.31237271411023443
then 		 0.1550627772357557
while 		 0.2558936679019712
although 		 0.2838967023889745
so 		 0.6209886629875147
because 		 0.17480744009448648
still 		 -0.04602102608868766
if 		 -0.14794185663476792
though 		 -0.05056378310731301


In [151]:
np.mean(list(counter.values()))

0.1494055003041596

In [205]:
from collections import defaultdict

In [221]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").ravel()

conpono_true = np.argwhere(conpono_pred == labels) 
bert_false = np.argwhere(bert_pred != labels) 
improvement = np.intersect1d(conpono_true, bert_false)

markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])

markers = set(["but", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])

from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
word2example = defaultdict(list)
print(len(improvement))
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(improvement):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[0] ])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
                    word2example[w].append([" ".join(sents[0]), " ".join(sents[1])])
            break
            
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
#     if j in indexes_to_skip:
#         continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1

counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc
np.average([c for w,c in counter.most_common()], weights=[general_word_counter[w] for w,c in counter.most_common()])

1241
100
200
300
400
500
600
700
800
900
1000
1100
1200
also 		 0.015531681979754594
when 		 0.3851946571513812
but 		 0.26822266388082017
after 		 0.009700351628191396
before 		 0.01910749776137334
although 		 0.37755220387054583
then 		 0.056852219900683454
while 		 -0.0298096621311726
so 		 0.48878312716444106
because 		 0.02913150593607541
though 		 0.20147410262393492
still 		 -0.0012746521938540017
if 		 -0.18471400179090133


0.12055290164812194

In [223]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").ravel()

conpono_true = np.argwhere(conpono_pred != labels) 
bert_false = np.argwhere(bert_pred == labels) 
regression = np.intersect1d(conpono_true, bert_false)

markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])

markers = set(["but", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])

from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
print(len(regression))
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(regression):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[1]])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
            break
            
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
#     if j in indexes_to_skip:
#         continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1

counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc
np.average([c for w,c in counter.most_common()], weights=[general_word_counter[w] for w,c in counter.most_common()])

799
100
200
300
400
500
600
700
also 		 0.24299598343426776
after 		 -0.012176749703440137
but 		 0.02340002636088041
when 		 -0.2691364636239045
while 		 0.3284519572953735
then 		 -0.015961513114537994
because 		 0.5970460688796844
although 		 0.01797084850220205
before 		 -0.43066344687341124
still 		 0.7712692763938316
so 		 0.5402341533859405
if 		 0.6869231203750777
though 		 0.16530873446962616


0.08176833661038796

In [213]:
for w in ["when", "but", "although"]:
    print(w)
    for e in random.sample(word2example[w], 4):
        print(e[0])
        print("-")
        print(e[1])
        print("")
    print("~~"*20)

when
Relevance is the concept of one topic being connected to another topic in a way that makes it useful to consider the second topic when considering the first .
-
The concept of relevance is studied in many different fields , including cognitive sciences , logic , and library and information science .

The skull remained unstudied until 1956 when Francis Fraser examined it and concluded that it was similar to species in both the genera `` Lagenorhynchus '' and `` Delphinus '' but not the same as either .
-
A new genus was created by simply merging these two names together .

Among the earliest references to music from Catalonia date to the Middle Ages , when Barcelona and the surrounding area were relatively prosperous , allowing both music and arts to be cultivated actively .
-
Catalonia and adjacent areas were the home for several troubadours , the itinerant composer-musicians whose influence and aesthetics was decisive on the formation of late medieval secular music , and who tra

Examples 

In 1941 Vaughn joined the United States National Guard for what had been planned as a one-year assignment , but when World War II broke out , he was sent abroad until the war ended in 1945 .

He decided to make music a career when he was discharged from the army at the end of the war , and attended Western Kentucky State College , now known as Western Kentucky University , majoring in music composition .

In [220]:
stats = []
for repetition in range(10):
    print(repetition)
    bert_pred = np.load("bert_predictions.npy").ravel()
    conpono_pred = np.load("conpono_predictions.npy").ravel()
    labels = np.load("labels.npy").ravel()

    random_samples = np.random.choice(len(labels), len(improvement), replace=False)

    markers = set(["but", "and", "as", "when", "if", "before", "because",
              "while", "though", "after", "so", "although", "then",
              "also", "still"])

    from collections import Counter
    all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
    all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
    all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

    word_counter = Counter()
    #print(len(random_samples))
    cc = 0
    indexes_to_skip = []
    num_words = 0
    for i in list(random_samples):
        # print(i)
        cc += 1
        if cc % 500 == 0:
            print(cc)
        bert_vec = se.evaluation.embed['test']['X'][i]
        for j in range(all_vecs.shape[0]):
            if (all_vecs[j] == bert_vec).all():
                sents = se.evaluation.data['test']['X'][j % 4000]
                words = set([w.lower() for w in sents[0] + sents[1]])
                num_words += len(words)
                indexes_to_skip.append(j)
                if words.intersection(markers):
                    for w in words.intersection(markers):
                        word_counter[w] += 1
                break

    general_word_counter = Counter()
    general_num_words = 0
    for j in range(len(se.evaluation.data['test']['X'])):
    #     if j in indexes_to_skip:
    #         continue
        sents = se.evaluation.data['test']['X'][j]
        words = set([w.lower() for w in sents[0] + sents[1]])
        general_num_words += len(words)
        if words.intersection(markers):
            for w in words.intersection(markers):
                general_word_counter[w] += 1

    counter = Counter()
    for w,c in word_counter.most_common():
        gc = general_word_counter[w]
        tmp_c = c / num_words
        tmp_gc = gc / general_num_words
        #print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
        counter[w] = (tmp_c - tmp_gc) / tmp_gc
    stats.append(np.average([c for w,c in counter.most_common()], weights=[general_word_counter[w] for w,c in counter.most_common()]))
print(np.mean(stats), np.std(stats))

0
500
1000
1
500
1000
2
500
1000
3
500
1000
4
500
1000
5
500
1000
6
500
1000
7
500
1000
8
500
1000
9
500
1000
0.00028860140393729505 0.009065909787388653


In [164]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").ravel()

conpono_true = np.argwhere(conpono_pred == labels) 
bert_false = np.argwhere(bert_pred != labels) 
improvement = np.intersect1d(conpono_true, bert_false)

markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])
from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(improvement):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[0]])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
            break
            
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
#     if j in indexes_to_skip:
#         continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1

counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc
np.mean(list(counter.values()))

100
200
300
400
500
600
700
800
900
1000
1100
1200
and 		 0.15935676489515455
as 		 -0.02397053388410366
also 		 0.015531681979754594
when 		 0.3851946571513812
but 		 0.26822266388082017
after 		 0.009700351628191396
before 		 0.01910749776137334
although 		 0.37755220387054583
then 		 0.056852219900683454
while 		 -0.0298096621311726
so 		 0.48878312716444106
because 		 0.02913150593607541
though 		 0.20147410262393492
still 		 -0.0012746521938540017
if 		 -0.18471400179090133


0.11807586178615496

In [171]:
print("Flipped", (np.array(indexes_to_skip) > 4000).sum())
print("Correct", (np.array(indexes_to_skip) < 4000).sum())

flip_counter = Counter()
for i in indexes_to_skip:
    if i < 4000:
        if (i + 4000) in indexes_to_skip:
            flip_counter["both"] += 1
        else:
            flip_counter["only_correct"] += 1
    else:
        if (i % 4000) not in indexes_to_skip:
            flip_counter["only_flipped"] += 1
print(flip_counter)

Flipped 961
Correct 280
Counter({'only_flipped': 854, 'only_correct': 173, 'both': 107})


In [172]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").ravel()

conpono_true = np.argwhere(conpono_pred != labels) 
bert_false = np.argwhere(bert_pred == labels) 
regression = np.intersect1d(conpono_true, bert_false)

markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])
from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(regression):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[1]])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
            break
            
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
#     if j in indexes_to_skip:
#         continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1

counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc
np.mean(list(counter.values()))

100
200
300
400
500
600
700
and 		 0.1933654153542571
as 		 -0.03433415103248922
also 		 0.24299598343426776
after 		 -0.012176749703440137
but 		 0.02340002636088041
when 		 -0.2691364636239045
while 		 0.3284519572953735
then 		 -0.015961513114537994
because 		 0.5970460688796844
although 		 0.01797084850220205
before 		 -0.43066344687341124
still 		 0.7712692763938316
so 		 0.5402341533859405
if 		 0.6869231203750777
though 		 0.16530873446962616


0.18697955067355723

In [173]:
print("Flipped", (np.array(indexes_to_skip) > 4000).sum())
print("Correct", (np.array(indexes_to_skip) < 4000).sum())

flip_counter = Counter()
for i in indexes_to_skip:
    if i < 4000:
        if (i + 4000) in indexes_to_skip:
            flip_counter["both"] += 1
        else:
            flip_counter["only_correct"] += 1
    else:
        if (i % 4000) not in indexes_to_skip:
            flip_counter["only_flipped"] += 1
print(flip_counter)

Flipped 167
Correct 632
Counter({'only_correct': 570, 'only_flipped': 105, 'both': 62})


In [181]:
bert_pred = np.load("bert_predictions.npy").ravel()
conpono_pred = np.load("conpono_predictions.npy").ravel()
labels = np.load("labels.npy").ravel()

pos_index = np.argwhere(0 == labels) 
conpono_true = np.argwhere(conpono_pred == labels) 
bert_false = np.argwhere(bert_pred != labels) 
improvement = np.intersect1d(conpono_true, bert_false)
improvement = np.intersect1d(improvement, pos_index)

markers = set(["but", "and", "as", "when", "if", "before", "because",
          "while", "though", "after", "so", "although", "then",
          "also", "still"])
from collections import Counter
all_vecs_p = np.load("../tmp/BSO-BSO wiki-conpono-test-x.npy")
all_vecs_n = np.load("../tmp/BSO-BSO wiki-conpono-test-y.npy")
all_vecs = np.concatenate((all_vecs_p, all_vecs_n) )

word_counter = Counter()
cc = 0
indexes_to_skip = []
num_words = 0
for i in list(improvement):
    # print(i)
    cc += 1
    if cc % 100 == 0:
        print(cc)
    bert_vec = se.evaluation.embed['test']['X'][i]
    for j in range(all_vecs.shape[0]):
        if (all_vecs[j] == bert_vec).all():
            sents = se.evaluation.data['test']['X'][j % 4000]
            words = set([w.lower() for w in sents[0] ])
            num_words += len(words)
            indexes_to_skip.append(j)
            if words.intersection(markers):
                for w in words.intersection(markers):
                    word_counter[w] += 1
            break
            
general_word_counter = Counter()
general_num_words = 0
for j in range(len(se.evaluation.data['test']['X'])):
#     if j in indexes_to_skip:
#         continue
    sents = se.evaluation.data['test']['X'][j]
    words = set([w.lower() for w in sents[0] + sents[1]])
    general_num_words += len(words)
    if words.intersection(markers):
        for w in words.intersection(markers):
            general_word_counter[w] += 1

counter = Counter()
for w,c in word_counter.most_common():
    gc = general_word_counter[w]
    tmp_c = c / num_words
    tmp_gc = gc / general_num_words
    print(w, "\t\t", (tmp_c - tmp_gc) / tmp_gc)
    counter[w] = (tmp_c - tmp_gc) / tmp_gc
np.mean(list(counter.values()))

100
200
300
400
500
600
700
800
900
and 		 0.16888267650442598
as 		 -0.04486665121424932
also 		 0.10528351397815239
after 		 0.045980802362786066
when 		 0.2131291384002924
but 		 0.0453525856646703
before 		 -0.05498259554867975
although 		 0.3517490331870735
then 		 0.02085213443815452
while 		 -0.04448240216588739
so 		 0.27828441181821106
because 		 -0.15654184302159027
still 		 0.10252030519320698
if 		 0.05001933827924476
though 		 -0.03287692526911669


0.06988690150711298

In [67]:
# DS Wiki

In [79]:
# Set up logger
PATH_TO_DATA = "../data/"
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)
config = BertConfig.from_pretrained('bert-{}-uncased'.format("base"))
config.output_hidden_states = True
config.output_attentions = True
tokenizer = BertTokenizer.from_pretrained('bert-{}-uncased'.format("base"))

# model = BertModel.from_pretrained('../models/conpono/bert/', config=config).cuda()
# model.eval()

# Set params for SentEval
params_discoeval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10, 'batch_size': 8,
                    'tokenizer': tokenizer, "layer": "pooler", "model": "base", "seed": 111}
params_discoeval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                 'tenacity': 5, 'epoch_size': 4}
se = discoeval.engine.SE(params_discoeval, batcher, prepare)


transfer_tasks = [
    ['SParxiv'],
    ['BSOwiki'],
    ['DCwiki'],
    ['PDTB-I']]
    #['SParxiv', 'SProc', 'SPwiki'], 
    #['DCchat', 'DCwiki'],
    #['BSOarxiv', 'BSOroc', 'BSOwiki'], 
    #['SSPabs', 'RST'],
    #[ 'PDTB-E', 'PDTB-I']]
    
#se.evaluation = SPEval(os.path.join(PATH_TO_DATA, 'SP/arxiv'), "Sentence Position arXiv", nclasses=5, seed=111)
#se.params.current_task = transfer_tasks[0][0]
#se.evaluation.do_prepare(se.params, se.prepare)
results = se.eval(transfer_tasks[3])
print(results)

2020-04-21 22:56:18,477 : Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-04-21 22:56:18,882 : https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-config.json HTTP/1.1" 200 0
2020-04-21 22:56:18,888 : loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/daniter/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
2020-04-21 22:56:18,890 : Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "outpu

DANITER False
DANITER cuda


2020-04-21 22:56:20,636 : [('reg:1e-09', 43.1)]
2020-04-21 22:56:20,637 : Validation : best param found is reg = 1e-09 with score             43.1
2020-04-21 22:56:20,638 : Evaluating...


DANITER False
DANITER cuda


2020-04-21 22:56:21,741 : Dev acc : 43.1 Test acc : 43.85 for PDTB



[[  0   3   9   0   0  10   1   0   7   0   0]
 [  0  73 148   0   0 166   7   8  31   1   0]
 [  0  15 539   0   0  90  25   0  97   7   0]
 [  0   0  11   0   0   0   0   0   2   0   0]
 [  0   1  14   0   1   3   0   0  10   0   0]
 [  0  25 130   0   0 325  11  14  33   1   0]
 [  0   2  62   0   0  23 152   0  54   0   0]
 [  0   2  12   0   0  43   3  14   3   1   0]
 [  0  10 262   0   0 117  45   2 202   3   0]
 [  0   9  29   0   0  57   1   0  20  20   0]
 [  0   2  11   0   0  38   2   2   3   0   0]]
{'PDTB-I': {'devacc': 43.1, 'acc': 43.85, 'ndev': 2972, 'ntest': 3024}}


In [102]:
148 - Comparison.Contrast (1), Contingency.Cause (2)
262 - Expansion.Restatement (8), Contingency.Cause (2)
166 - Comparison.Contrast (1) , Expansion.Conjunction (5)
130 - Expansion.Conjunction (5), Contingency.Cause (2)

SyntaxError: invalid syntax (<ipython-input-102-3e2c8d916d4a>, line 2)

In [81]:
from sklearn.metrics import confusion_matrix

In [82]:
preds = np.load("./pdtb_preds.npy")

In [83]:
len(preds)

3024

In [93]:
len((2)[1])

3024

In [94]:
labels = se.evaluation.data['test'][2]

In [101]:
cm = confusion_matrix(labels, preds)
np.fill_diagonal(cm, 0)
cm

array([[  0,   3,   9,   0,   0,  10,   1,   0,   7,   0,   0],
       [  0,   0, 148,   0,   0, 166,   7,   8,  31,   1,   0],
       [  0,  15,   0,   0,   0,  90,  25,   0,  97,   7,   0],
       [  0,   0,  11,   0,   0,   0,   0,   0,   2,   0,   0],
       [  0,   1,  14,   0,   0,   3,   0,   0,  10,   0,   0],
       [  0,  25, 130,   0,   0,   0,  11,  14,  33,   1,   0],
       [  0,   2,  62,   0,   0,  23,   0,   0,  54,   0,   0],
       [  0,   2,  12,   0,   0,  43,   3,   0,   3,   1,   0],
       [  0,  10, 262,   0,   0, 117,  45,   2,   0,   3,   0],
       [  0,   9,  29,   0,   0,  57,   1,   0,  20,   0,   0],
       [  0,   2,  11,   0,   0,  38,   2,   2,   3,   0,   0]])

In [113]:
c = 0
for i, label  in enumerate(labels):

    if label == 8 and preds[i] == 2:
        c += 1
        s1 = " ".join(se.evaluation.data['test'][0][i])
        s2 = " ".join(se.evaluation.data['test'][1][i])
        print(s1)
        print(s2)
        print("#"*40)
    if c > 5:
        break
        

It gets more mail in a month than McCall 's got in a year , and it 's not from mothers
I feel about Sassy like I did about Working Woman 10 years ago
########################################
In addition , McCall 's put in a less than stellar performance
Until a recent comeback , it saw steep losses in ad pages and circulation
########################################
his magazines will offer what many women 's magazines do n't
We write straight for women on their level
########################################
We write straight for women on their level
We do n't have passive readers
########################################
We do n't have passive readers
that even Success , in part , fits the company 's image , since about 30 % of its readership is female
########################################
Besides , we have enough on our plate
There is plenty of work to be done on what we have
########################################
